In [4]:
%pip install python-dotenv
# Install LangChain and required packages
%pip install langchain langchain_community
%pip install -U langchain-ollama
%pip install -qU langchain-openai
%pip install chromadb
%pip install pydantic
%pip install kagglehub
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("blastchar/telco-customer-churn")

print("Path to dataset files:", path)

/Users/arecalde/study/USD/AAI-510/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 172k/172k [00:00<00:00, 2.57MB/s]

Extracting files...
Path to dataset files: /Users/arecalde/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1


In [ ]:
# True will use ollama, False will use OpenAI
LOCAL = True

In [5]:
import pandas as pd
import os

# Find the CSV file in the downloaded path
data_dir = path
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print("CSV files found:", csv_files)

# Load the main dataset (usually 'WA_Fn-UseC_-Telco-Customer-Churn.csv')
df = pd.read_csv(os.path.join(data_dir, csv_files[0]))
print("DataFrame shape:", df.shape)
df.head()

CSV files found: ['WA_Fn-UseC_-Telco-Customer-Churn.csv']
DataFrame shape: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [47]:
from pydantic import BaseModel

# Define a Pydantic model for user features
class UserFeatures(BaseModel):
    MonthlyCharges: float
    tenure: int
    Contract: str

# Update your simulated model functions to accept a UserFeatures instance
def logistic_regression_predict(user_features: UserFeatures):
    return "Churn" if user_features.MonthlyCharges > 70 else "No Churn"

def random_forest_predict(user_features: UserFeatures):
    return "Churn" if user_features.tenure < 12 else "No Churn"

def xgboost_predict(user_features: UserFeatures):
    return "Churn" if user_features.Contract == "Month-to-month" else "No Churn"

# Example usage
user = UserFeatures(MonthlyCharges=80, tenure=5, Contract="Month-to-month")
print("Logistic Regression:", logistic_regression_predict(user))
print("Random Forest:", random_forest_predict(user))
print("XGBoost:", xgboost_predict(user))

Logistic Regression: Churn
Random Forest: Churn
XGBoost: Churn


In [75]:
import json

def parse_user_features(user_features_str):
    # Remove leading/trailing whitespace
    user_features_str = user_features_str.strip()
    # If string starts and ends with a single or double quote, strip them
    if (user_features_str.startswith("'") and user_features_str.endswith("'")) or \
       (user_features_str.startswith('"') and user_features_str.endswith('"')):
        user_features_str = user_features_str[1:-1]
    # Now try to parse
    try:
        return json.loads(user_features_str)
    except Exception:
        # If it fails, return as is (will error later)
        return user_features_str

def logistic_regression_predict_wrapper(user_features_str: str):
    print("Wrapper received:", repr(user_features_str))
    try:
        features = parse_user_features(user_features_str)
        user_features = UserFeatures(**features)
        return logistic_regression_predict(user_features)
    except Exception as e:
        return f"Error: {e}"

def random_forest_predict_wrapper(user_features_str: str):
    print("Wrapper received:", repr(user_features_str))
    try:
        features = parse_user_features(user_features_str)
        user_features = UserFeatures(**features)
        return random_forest_predict(user_features)
    except Exception as e:
        return f"Error: {e}"

def xgboost_predict_wrapper(user_features_str: str):
    print("Wrapper received:", repr(user_features_str))
    try:
        features = parse_user_features(user_features_str)
        user_features = UserFeatures(**features)
        return xgboost_predict(user_features)
    except Exception as e:
        return f"Error: {e}"

In [71]:
from langchain.tools import StructuredTool

logistic_regression_tool = StructuredTool.from_function(
    func=logistic_regression_predict_wrapper,
    name="LogisticRegressionChurnPredictor",
    description="Use ONLY to predict churn for a specific user using logistic regression. Input must be a JSON string with user features."
)

random_forest_tool = StructuredTool.from_function(
    func=random_forest_predict_wrapper,
    name="RandomForestChurnPredictor",
    description="Use ONLY to predict churn for a specific user using random forest. Input must be a JSON string with user features."
)

xgboost_tool = StructuredTool.from_function(
    func=xgboost_predict_wrapper,
    name="XGBoostChurnPredictor",
    description="Use ONLY to predict churn for a specific user using XGBoost. Input must be a JSON string with user features."
)

# Organize all model tools into a list for easy access
model_tools = [
    logistic_regression_tool,
    random_forest_tool,
    xgboost_tool
]

# Test with a JSON string
test_user = '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
for tool in model_tools:
    print(f"{tool.name}: {tool(test_user)}")

Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
LogisticRegressionChurnPredictor: Churn
Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
RandomForestChurnPredictor: Churn
Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
XGBoostChurnPredictor: Churn


In [66]:
# Organize all model tools into a list for easy access
model_tools = [
    logistic_regression_tool,
    random_forest_tool,
    xgboost_tool
]

# Test with a JSON string
test_user = '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
for tool in model_tools:
    print(f"{tool.name}: {tool(test_user)}")

Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
LogisticRegressionChurnPredictor: Churn
RandomForestChurnPredictor: Churn
XGBoostChurnPredictor: Churn


In [67]:
from langchain.docstore.document import Document

# Dummy knowledge base with EDA and model info
knowledge_base = """
Exploratory Data Analysis (EDA) Summary:
- The dataset contains 7,043 customer records with 21 features each.
- Average customer tenure is 32 months.
- 26.5% of customers have churned.
- Customers with month-to-month contracts have a higher churn rate.
- Senior citizens are more likely to churn than younger customers.

Model Training Information:
- Logistic Regression: Trained with L2 regularization, C=1.0, max_iter=1000. Precision: 0.78, Recall: 0.65, F1: 0.71.
- Random Forest: 100 trees, max_depth=8, min_samples_split=4. Precision: 0.81, Recall: 0.69, F1: 0.74.
- XGBoost: 200 rounds, learning_rate=0.1, max_depth=6. Precision: 0.83, Recall: 0.72, F1: 0.77.
- All models used stratified 80/20 train-test split and standard scaling for numeric features.
- The most important features for predicting churn were tenure, MonthlyCharges, and Contract type.
"""

docs = [Document(page_content=knowledge_base)]

In [68]:
from langchain.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import ChatOllama

# Choose embedding and LLM based on LOCAL
if LOCAL:
    embedding = OllamaEmbeddings(model="llama3.2")
    llm = ChatOllama(model="llama3.2")
else:
    embedding = OpenAIEmbeddings()
    llm = ChatOpenAI(model="gpt-3.5-turbo")

# The rest of your code remains the same
# CharacterTextSplitter, Chroma, and RetrievalQA are already imported and available in this notebook.
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(split_docs, embedding)
retriever = vectorstore.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)
query = "What was the average customer tenure? How well did the XGBoost model perform?"
response = rag_chain.invoke(query)
print("RAG Answer:", response["result"])


RAG Answer: I don't know the average customer tenure.

As for the XGBoost model performance, according to the provided information, its Precision was 0.83, Recall was 0.72, and F1 score was 0.77.


In [76]:
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

# 2. Register all tools (RAG + prediction tools)
all_tools = [
    logistic_regression_tool,
    random_forest_tool,
    xgboost_tool
]

# 3. Create the agent
system_message = (
    "If the question asks for predictions from all models, call each prediction tool and report all results. "
    "If the question is about predicting churn for a user with a specific model, use the appropriate tool. "
    "If both, answer both parts using the appropriate tools. "
    "If you do not know the answer and the available tools are not sufficient, just say 'I don't know'. "
    "Do not attempt to make up an answer."
)

agent_executor = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    system_message=system_message,
    max_iterations=3
)

# 4. Ask the agent a question
user_query = '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'
response = agent_executor.invoke(user_query)
print("\nAgent Answer:")
print(response["output"])



> Entering new AgentExecutor chain...
Action: LogisticRegressionChurnPredictor
Action Input: {"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'

Observation: Churn
Thought:Question: What is the predicted churn probability for a user with monthly charges of $80, tenure of 5 months, and a month-to-month contract?

Action: RandomForestChurnPredictor
Action Input: {"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'

Observation: Churn
Thought:Question: What is the predicted churn probability for a user with monthly charges of $80, tenure of 5 months, and a month-to-month contract?

Action: XGBoostChurnPredictor
Action Input: {"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}Wrapper received: '{"MonthlyCharges": 80, "tenure": 5, "Contract": "Month-to-month"}'

Observation: Churn